# Creating required Files for YOLO

In [0]:
import os
import time
import cv2
import numpy as np

In [0]:
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd

In [0]:
annotations = glob('dataset-master/Annotations/*.xml')

In [0]:
dff = []

In [0]:
cnt = 0
for file in annotations:
    filename = file.split('\\')[-1]
    filename =filename.split('.')[0] + '.jpg'
    row = []
    parsedXML = ET.parse(file)
    for node in parsedXML.getroot().iter('object'):
        blood_cells = node.find('name').text
        xmin = int(node.find('bndbox/xmin').text)
        xmax = int(node.find('bndbox/xmax').text)
        ymin = int(node.find('bndbox/ymin').text)
        ymax = int(node.find('bndbox/ymax').text)

        row = [filename, blood_cells, xmin, xmax, ymin, ymax]
        dff.append(row)
        cnt += 1

In [0]:
df = pd.DataFrame(dff, columns=['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

In [0]:
df.head()

,filename,cell_type,xmin,xmax,ymin,ymax
0,dataset-master/Annotations/BloodImage_00335.jpg,RBC,220,326,222,315
1,dataset-master/Annotations/BloodImage_00335.jpg,RBC,363,471,4,122
2,dataset-master/Annotations/BloodImage_00335.jpg,RBC,422,507,150,252
3,dataset-master/Annotations/BloodImage_00335.jpg,RBC,256,362,4,109
4,dataset-master/Annotations/BloodImage_00335.jpg,RBC,116,261,128,238


In [0]:
df["width"] = abs(df["xmax"] - df["xmin"])
df["height"] = abs(df["ymax"] - df["ymin"])

In [0]:
df.tail()

,filename,cell_type,xmin,xmax,ymin,ymax,width,height
4883,dataset-master/Annotations/BloodImage_00312.jpg,WBC,78,323,275,480,245,205
4884,dataset-master/Annotations/BloodImage_00312.jpg,Platelets,326,367,186,217,41,31
4885,dataset-master/Annotations/BloodImage_00312.jpg,Platelets,339,374,213,253,35,40
4886,dataset-master/Annotations/BloodImage_00312.jpg,Platelets,370,406,212,247,36,35
4887,dataset-master/Annotations/BloodImage_00312.jpg,Platelets,269,309,294,328,40,34


In [0]:
a = df["filename"].str.split("/")

In [0]:
b=[]
for i in range(0,len(a)):
  b.append(a[i][2])

In [0]:
df["file"] = b

In [0]:
df.tail(20)

,filename,cell_type,xmin,xmax,ymin,ymax,width,height,file
4868,dataset-master/Annotations/BloodImage_00360.jpg,RBC,205,313,59,147,108,88,BloodImage_00360.jpg
4869,dataset-master/Annotations/BloodImage_00360.jpg,RBC,529,628,238,331,99,93,BloodImage_00360.jpg
4870,dataset-master/Annotations/BloodImage_00360.jpg,RBC,220,334,377,479,114,102,BloodImage_00360.jpg
4871,dataset-master/Annotations/BloodImage_00360.jpg,RBC,133,232,394,480,99,86,BloodImage_00360.jpg
4872,dataset-master/Annotations/BloodImage_00360.jpg,RBC,329,456,129,224,127,95,BloodImage_00360.jpg
4873,dataset-master/Annotations/BloodImage_00360.jpg,RBC,291,400,1,84,109,83,BloodImage_00360.jpg
4874,dataset-master/Annotations/BloodImage_00360.jpg,WBC,246,477,211,415,231,204,BloodImage_00360.jpg
4875,dataset-master/Annotations/BloodImage_00360.jpg,Platelets,564,596,333,372,32,39,BloodImage_00360.jpg
4876,dataset-master/Annotations/BloodImage_00312.jpg,RBC,98,197,203,294,99,91,BloodImage_00312.jpg
4877,dataset-master/Annotations/BloodImage_00312.jpg,RBC,172,271,184,275,99,91,BloodImage_00312.jpg


In [0]:
df.to_csv("data.csv")

In [0]:
df = pd.read_csv("data.csv")

In [0]:
df.head()

,Unnamed: 0,filename,cell_type,xmin,xmax,ymin,ymax,width,height,file
0,0,dataset-master/Annotations/BloodImage_00335.jpg,RBC,220,326,222,315,106,93,BloodImage_00335.jpg
1,1,dataset-master/Annotations/BloodImage_00335.jpg,RBC,363,471,4,122,108,118,BloodImage_00335.jpg
2,2,dataset-master/Annotations/BloodImage_00335.jpg,RBC,422,507,150,252,85,102,BloodImage_00335.jpg
3,3,dataset-master/Annotations/BloodImage_00335.jpg,RBC,256,362,4,109,106,105,BloodImage_00335.jpg
4,4,dataset-master/Annotations/BloodImage_00335.jpg,RBC,116,261,128,238,145,110,BloodImage_00335.jpg


In [0]:
df["file"].nunique()

364

In [0]:
a=df["file"].unique()

In [0]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()

In [0]:
le.fit(df["cell_type"])

LabelEncoder()

In [0]:
b=le.transform(df["cell_type"])

In [0]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(a , test_size=0.2, random_state=1250)

In [0]:
df["classes"]=b

In [0]:
df.head()

,Unnamed: 0,filename,cell_type,xmin,xmax,ymin,ymax,width,height,file,classes
0,0,dataset-master/Annotations/BloodImage_00335.jpg,RBC,220,326,222,315,106,93,BloodImage_00335.jpg,1
1,1,dataset-master/Annotations/BloodImage_00335.jpg,RBC,363,471,4,122,108,118,BloodImage_00335.jpg,1
2,2,dataset-master/Annotations/BloodImage_00335.jpg,RBC,422,507,150,252,85,102,BloodImage_00335.jpg,1
3,3,dataset-master/Annotations/BloodImage_00335.jpg,RBC,256,362,4,109,106,105,BloodImage_00335.jpg,1
4,4,dataset-master/Annotations/BloodImage_00335.jpg,RBC,116,261,128,238,145,110,BloodImage_00335.jpg,1


In [0]:
a[0]

'BloodImage_00335.jpg'

In [0]:
df['xcenter'] = (df['xmin'] + df['xmax'])/2
df['ycenter'] = (df['ymin'] + df['ymax'])/2

In [0]:
df.head()

,Unnamed: 0,filename,cell_type,xmin,xmax,ymin,ymax,width,height,file,classes,xcenter,ycenter
0,0,dataset-master/Annotations/BloodImage_00335.jpg,RBC,220,326,222,315,106,93,BloodImage_00335.jpg,1,273.0,268.5
1,1,dataset-master/Annotations/BloodImage_00335.jpg,RBC,363,471,4,122,108,118,BloodImage_00335.jpg,1,417.0,63.0
2,2,dataset-master/Annotations/BloodImage_00335.jpg,RBC,422,507,150,252,85,102,BloodImage_00335.jpg,1,464.5,201.0
3,3,dataset-master/Annotations/BloodImage_00335.jpg,RBC,256,362,4,109,106,105,BloodImage_00335.jpg,1,309.0,56.5
4,4,dataset-master/Annotations/BloodImage_00335.jpg,RBC,116,261,128,238,145,110,BloodImage_00335.jpg,1,188.5,183.0


In [0]:
len(train)

291

In [0]:
newdf = pd.DataFrame()

In [0]:
newdf['classes'] = df['classes']
newdf['file'] = df['file']

newdf['xcenter'] = round( df['xcenter'] / 640   , 6 )
newdf['ycenter'] = round( df['ycenter'] / 480   , 6 )

newdf['width'] = round( df['width'] / 640     ,6 )
newdf['height'] = round( df['height'] / 480 , 6 )

In [0]:
newdf.head(15)

,classes,file,xcenter,ycenter,width,height
0,1,BloodImage_00335.jpg,0.426562,0.559375,0.165625,0.193750
1,1,BloodImage_00335.jpg,0.651562,0.131250,0.168750,0.245833
2,1,BloodImage_00335.jpg,0.725781,0.418750,0.132812,0.212500
3,1,BloodImage_00335.jpg,0.482812,0.117708,0.165625,0.218750
4,1,BloodImage_00335.jpg,0.294531,0.381250,0.226562,0.229167
5,1,BloodImage_00335.jpg,0.682031,0.706250,0.179688,0.212500
6,1,BloodImage_00335.jpg,0.194531,0.563542,0.120312,0.160417
7,1,BloodImage_00335.jpg,0.174219,0.881250,0.154688,0.195833
8,1,BloodImage_00335.jpg,0.242969,0.177083,0.170312,0.233333
9,1,BloodImage_00335.jpg,0.475000,0.367708,0.168750,0.210417


In [0]:
newdf.iloc[:,[0,2,3,4,5]].to_csv("Yolo_annt.csv",index=False)

In [8]:
for z in a:
  t=newdf[newdf["file"] == z].iloc[:,[0,2,3,4,5]].values
  s=''
  for i in range(0,len(t)): 
    t1 = ','.join(str(x) for x in t[i])
    if(i < len(t)-1):
      s += t1 + '\n'
    else:
      s=s+t1

  
  spl = z.split('.')
  fn = "data/Images/" +spl[0]+ ".txt" 
  print(fn)
  f = open(fn,'w+')
  f.write(s)
  f.close()

data/Images/BloodImage_00335.txt
data/Images/BloodImage_00346.txt
data/Images/BloodImage_00319.txt
data/Images/BloodImage_00064.txt
data/Images/BloodImage_00290.txt
data/Images/BloodImage_00115.txt
data/Images/BloodImage_00293.txt
data/Images/BloodImage_00258.txt
data/Images/BloodImage_00124.txt
data/Images/BloodImage_00320.txt
data/Images/BloodImage_00278.txt
data/Images/BloodImage_00310.txt
data/Images/BloodImage_00016.txt
data/Images/BloodImage_00159.txt
data/Images/BloodImage_00235.txt
data/Images/BloodImage_00126.txt
data/Images/BloodImage_00383.txt
data/Images/BloodImage_00396.txt
data/Images/BloodImage_00012.txt
data/Images/BloodImage_00385.txt
data/Images/BloodImage_00219.txt
data/Images/BloodImage_00223.txt
data/Images/BloodImage_00336.txt
data/Images/BloodImage_00024.txt
data/Images/BloodImage_00086.txt
data/Images/BloodImage_00268.txt
data/Images/BloodImage_00141.txt
data/Images/BloodImage_00018.txt
data/Images/BloodImage_00176.txt
data/Images/BloodImage_00384.txt
data/Image

In [0]:
t=df[df["file"] == a[0]].iloc[:,[10,3,4,5,6]].values

'1,220,326,222,315'

In [0]:
d=[]
for i in range(0,len(a)):
    t=df[df["file"] == a[i]].iloc[:,[10,3,4,5,6]].values
    tt=t.ravel().tolist()
    values = ','.join(str(v) for v in tt)
    string = a[i] +" "+ values
    d.append(string)  

In [0]:
t=df[df["file"] == "BloodImage_00335.jpg"].iloc[:,[10,3,4,5,6]].values
tt=t.ravel().tolist()
values = ','.join(str(v) for v in tt)

'1,220,326,222,315,1,363,471,4,122,1,422,507,150,252,1,256,362,4,109,1,116,261,128,238,1,379,494,288,390,1,86,163,232,309,1,62,161,376,470,1,101,210,29,141,1,250,358,126,227,1,175,272,368,480,2,265,490,334,480'

In [0]:
d

['BloodImage_00335.jpg 1,220,326,222,315,1,363,471,4,122,1,422,507,150,252,1,256,362,4,109,1,116,261,128,238,1,379,494,288,390,1,86,163,232,309,1,62,161,376,470,1,101,210,29,141,1,250,358,126,227,1,175,272,368,480,2,265,490,334,480',
 'BloodImage_00346.jpg 1,403,502,68,168,1,138,245,201,303,1,288,370,105,200,1,320,425,157,278,1,222,309,40,143,1,409,536,244,354,1,489,616,99,219,1,35,168,156,234,1,16,117,365,459,1,36,121,1,86,1,138,238,45,159,1,319,432,1,79,1,566,640,348,463,2,162,399,274,480',
 'BloodImage_00319.jpg 1,416,512,371,478,1,460,566,90,195,1,414,532,245,340,1,188,278,274,374,1,117,216,344,440,1,148,250,392,470,1,309,418,65,168,1,403,493,265,359,1,565,638,117,224,1,555,640,39,132,2,72,327,1,209,0,591,632,391,429,0,36,74,233,262',
 'BloodImage_00064.jpg 1,474,609,202,309,1,356,441,248,333,1,393,494,343,429,1,232,334,150,240,1,416,518,1,90,1,204,306,45,135,1,218,320,316,406,1,507,612,308,399,1,501,603,406,480,1,1,85,228,308,2,28,172,129,243',
 'BloodImage_00290.jpg 1,222,332,343

In [0]:
df[df["file"] == "BloodImage_00267.jpg"].iloc[:,[10,3,4,5,6]]

,classes,xmin,xmax,ymin,ymax
4473,2,87,298,112,298
4474,0,62,101,26,73
4475,0,15,53,309,350
4476,0,1,38,349,381


In [0]:
pd.DataFrame(d , columns=["annot"]).to_csv("Yolo_annt.csv",index=False)